In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# from HelperFunctions import minibatch 
%reload_ext autoreload
%autoreload 2
from HelperFunctions import minibatch, dummify_columns, undummify, feature_standardize, label_encode_column, columns_of_type

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn import metrics

In [2]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, AdaBoostRegressor
randomForest = RandomForestRegressor()
gbm = GradientBoostingRegressor()
abr = AdaBoostRegressor()
import xgboost as xgb
import lightgbm as lgb

In [93]:
df=pd.read_csv('down_sampled_df.csv')

In [51]:
df.earliest_cr_line=pd.to_datetime(df.earliest_cr_line)
df.earliest_cr_line=df.earliest_cr_line.apply(lambda x:x.year)
df.earliest_cr_line=df.earliest_cr_line.astype('int')

In [52]:
drop_list=['id','sub_grade','issue_d','loan_status','zip_code','addr_state','RANDOM']
x_train=df.drop(drop_list,axis=1)
y_train=df.loan_status

In [54]:
cat_list=columns_of_type(x_train,'string')
cont_list=columns_of_type(x_train,'number')

In [55]:
dum_df=dummify_columns(x_train,cat_list)

In [58]:
final_df=dum_df.apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)

In [59]:
final_df.head()

,funded_amnt,term,int_rate,annual_inc,dti,delinq_2yrs,earliest_cr_line,mths_since_last_delinq,mths_since_last_derog_record,derog_records,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,collection_amt,install_util,all_util,rev_credit,inq_last_12m,chargeoff_within_12_mths,delinq_amnt,mths_since_RecentAcc_opened,all_accs_120days+_PastDue_ever,accs_90days+_PastDue_24m,accs_opened_past_12m,pct_acc_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_credit,install_credit,fico,Outstanding_mortgage_debt,revol_frac,install_frac,mort_frac,card_frac,active_card_frac,active_revol_frac,active_install_frac,open_revol_frac,good_acc_frac,loan_duration,profit,grade__B,grade__C,grade__D,grade__E,grade__F,grade__G,emp_length__1 year,emp_length__10+ years,emp_length__2 years,emp_length__3 years,emp_length__4 years,emp_length__5 years,emp_length__6 years,emp_length__7 years,emp_length__8 years,emp_length__9 years,emp_length__< 1 year,home_ownership__MORTGAGE,home_ownership__NONE,home_ownership__OTHER,home_ownership__OWN,home_ownership__RENT,verification_status__Source Verified,verification_status__Verified,purpose__credit_card,purpose__debt_consolidation,purpose__educational,purpose__home_improvement,purpose__house,purpose__major_purchase,purpose__medical,purpose__moving,purpose__other,purpose__renewable_energy,purpose__small_business,purpose__vacation,purpose__wedding,initial_list_status__w,application_type__Joint App
0,1.096593,-0.744549,-1.578352,0.302359,-0.011889,-0.373743,-0.291019,0.948916,-0.486075,-0.345833,0.079528,0.431540,-0.125591,-0.631490,-0.07037,-0.027296,-0.752786,-0.839942,0.293693,-0.840292,-0.087094,-0.018,0.203234,0.215566,0.217722,0.211349,0.230898,-0.376109,-0.117111,0.560668,0.435132,0.427199,0.446205,0.218354,0.217887,0.182545,0.218297,0.224541,0.217793,-0.819481,0.218014,0.196868,1.226656,1.427811,-0.493285,-0.684521,-0.54245,-0.393604,-0.239098,-0.131661,-0.2645,-0.688152,-0.313926,-0.293852,-0.252882,-0.259234,-0.217438,-0.213872,-0.218051,-0.197092,3.356900,1.081505,-0.00612,-0.01224,-0.349822,-0.868004,1.239265,-0.758213,-0.479244,0.804623,-0.014353,-0.252924,-0.080752,-0.145354,-0.11263,-0.092286,-0.258039,-0.028054,-0.130553,-0.084654,-0.041541,0.873588,-0.147223
1,-0.135562,-0.744549,-1.781998,-0.395669,-0.996114,-0.373743,-1.494084,-1.033964,-0.486075,-0.345833,-0.802855,-1.296677,-0.125591,-0.631490,-0.07037,-0.070450,-0.752786,-0.839942,1.138573,-0.840292,-0.087094,-0.018,0.346300,0.215566,0.217722,0.206555,0.245363,-0.376109,-0.117111,-0.555089,-0.936423,3.253364,-0.597713,0.219048,0.216605,0.183238,0.219633,0.225177,0.218082,-0.819481,0.217292,0.195989,1.473043,0.238298,-0.493285,-0.684521,-0.54245,-0.393604,-0.239098,-0.131661,-0.2645,-0.688152,-0.313926,3.403075,-0.252882,-0.259234,-0.217438,-0.213872,-0.218051,-0.197092,-0.297894,-0.924638,-0.00612,-0.01224,-0.349822,1.152068,-0.806930,-0.758213,2.086620,-1.242818,-0.014353,-0.252924,-0.080752,-0.145354,-0.11263,-0.092286,-0.258039,-0.028054,-0.130553,-0.084654,-0.041541,0.873588,-0.147223
2,-1.143688,-0.744549,-2.101722,0.102922,-0.795841,-0.373743,1.179395,-1.033964,-0.486075,-0.345833,-0.844873,0.596133,-0.125591,-0.631490,-0.07037,-0.070450,-0.752786,-0.839942,1.554088,-0.840292,-0.087094,-0.018,0.184159,0.215566,0.217722,0.216144,0.245363,-0.376109,-0.117111,0.357639,-0.240537,1.369254,0.187647,0.218314,0.218257,0.182155,0.218638,0.225356,0.218185,-0.819481,0.218836,0.197693,1.226656,-0.722955,-0.493285,-0.684521,-0.54245,-0.393604,-0.239098,-0.131661,-0.2645,-0.688152,-0.313926,-0.293852,-0.252882,3.857520,-0.217438,-0.213872,-0.218051,-0.197092,-0.297894,1.081505,-0.00612,-0.01224,-0.349822,-0.868004,-0.806930,-0.758213,-0.479244,0.804623,-0.014353,-0.252924,-0.080752,-0.145354,-0.11263,-0.092286,-0.258039,-0.028054,-0.130553,-0.084654,-0.041541,0.873588,-0.147223
3,-0.975667,-0.744549,-1.619081,-0.167742,-0.501746,-0.373743,0.377351,1.032181,-0.486075,-0.345833,-0.537615,-0.802901,-0.125591,1.634524,-0.07037,0.32

In [61]:
logit=LogisticRegression(C=1e4)
logit.fit(final_df,y_train)

/home/auscheng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=10000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [62]:
logit.score(final_df,y_train)

0.969104594989327

In [111]:
prob=logit.predict_proba(final_df) #[Default,Fully Paid]


In [110]:
def adjust_prob(unsampled_df,sampled_df,sampled_prob): # sampled_df, sampled_prob can also be test_df, test_prob
    # turning the predicted probability into a dataframe with column name default_prob
    sample_prob_df=pd.DataFrame(sample_prob[:,0],columns=['default_prob'])
    # find actual default rate for each class
    grade_rate=unsampled_df.groupby('grade')['loan_status'].apply(lambda x:(x=='Default').sum()/x.count())
    grade_rate_dict=grade_rate.to_dict()
    # mapping the unsampled_df default rates to the test_df=sampled_df
    # and then getting the array of default_rates in the test_df
    sampled_df['default_rate']=sampled_df['grade'].map(grade_rate_dict)
    original_fraction=sampled_df['default_rate'].values
    # Adjusting the default_probability to the true probability (accounting for down/up sampling)    
    sampled_fraction=0.5    
    prob=1/(1+(1/original_fraction-1)/(1/sampled_fraction-1)*(1/sampled_prob-1))
    return prob

In [113]:
adjust_prob(df,df,prob)

/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in true_divide
  del sys.path[0]


ValueError: operands could not be broadcast together with shapes (53406,) (53406,2) 

In [79]:
grade_rate=df.groupby('grade')['loan_status'].apply(lambda x:(x=='Default').sum()/x.count())

In [95]:
grade_rate_dict=grade_rate.to_dict()

In [109]:
df['default_rate']=df['grade'].map(grade_rate_dict)
prob_df=pd.concat([df,sample_prob_df],axis=1)
prob_df['default_rate'].values

array([0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5])

In [107]:
prob_df

,id,funded_amnt,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,mths_since_last_delinq,mths_since_last_derog_record,derog_records,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,acc_now_delinq,collection_amt,install_util,all_util,rev_credit,inq_last_12m,chargeoff_within_12_mths,delinq_amnt,mths_since_RecentAcc_opened,all_accs_120days+_PastDue_ever,accs_90days+_PastDue_24m,accs_opened_past_12m,pct_acc_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_credit,install_credit,fico,Outstanding_mortgage_debt,revol_frac,install_frac,mort_frac,card_frac,active_card_frac,active_revol_frac,active_install_frac,open_revol_frac,good_acc_frac,loan_duration,profit,RANDOM,default_rate,default_prob
0,65251140,25000.0,36,7.89,A,A5,< 1 year,MORTGAGE,94000.0,Source Verified,2015-11-01,Fully Paid,debt_consolidation,933xx,CA,19.36,0.0,1997-09-01,25.0,-999.0,0.0,57.1,30.0,w,0.0,-999.0,Individual,0.0,121.0,-999.0,-999.0,36500.0,-999.0,0.0,0.0,4.0,0.0,0.0,1.0,96.7,0.0,0.0,239880.0,58955.0,702.0,131064.0,0.633333,0.266667,0.100000,0.578947,0.363636,0.368421,-999.000000,0.578947,0.500000,34.500000,28136.954317,0.220221,0.5,0.033570
1,62519711,14000.0,36,6.89,A,A3,3 years,RENT,45000.0,Not Verified,2015-10-01,Fully Paid,credit_card,288xx,NC,8.45,0.0,1988-12-01,-999.0,-999.0,0.0,23.5,9.0,w,0.0,-999.0,Individual,0.0,0.0,-999.0,-999.0,60900.0,-999.0,0.0,0.0,34.0,0.0,0.0,0.0,100.0,0.0,0.0,60900.0,0.0,777.0,0.0,0.777778,0.000000,0.222222,0.857143,0.500000,0.428571,-999.000000,0.428571,0.333333,37.566667,15571.536315,0.008344,0.5,0.114899
2,50646937,5000.0,36,5.32,A,A1,5 years,MORTGAGE,80000.0,Not Verified,2015-05-01,Fully Paid,debt_consolidation,483xx,MI,10.67,0.0,2008-02-01,-999.0,-999.0,0.0,21.9,32.0,w,0.0,-999.0,Individual,0.0,0.0,-999.0,-999.0,72900.0,-999.0,0.0,0.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,207312.0,29912.0,727.0,98602.0,0.625000,0.343750,0.031250,0.650000,0.538462,0.450000,-999.000000,0.750000,0.656250,34.500000,5417.327585,0.412407,0.5,0.181143
3,16442318,6500.0,36,7.69,A,A4,7 years,MORTGAGE,61000.0,Not Verified,2014-05-01,Fully Paid,debt_consolidation,985xx,WA,13.93,0.0,2002-10-01,68.0,-999.0,0.0,33.6,15.0,w,0.0,68.0,Individual,0.0,1101.0,-999.0,-999.0,4200.0,-999.0,0.0,0.0,13.0,4.0,0.0,0.0,73.3,0.0,0.0,217544.0,27075.0,662.0,178608.0,0.400000,0.466667,0.133333,0.500000,0.666667,0.500000,-999.000000,0.666667,0.600000,36.533333,7299.283210,0.562369,0.5,0.226167
4,49924755,16000.0,36,5.32,A,A1,10+ years,MORTGAGE,98000.0,Source Verified,2015-06-01,Fully Paid,credit_card,305xx,GA,4.69,1.0,1993-04-01,18.0,-999.0,0.0,27.7,42.0,w,0.0,-999.0,Individual,0.0,0.0,-999.0,-999.0,60100.0,-999.0,0.0,0.0,10.0,0.0,0.0,4.0,97.6,0.0,0.0,254900.0,0.0,747.0,179982.0,0.714286,0.142857,0.119048,0.533333,0.187500,0.166667,-999.000000,0.433333,0.333333,36.533333,17336.643738,0.941376,0.5,0.271506
5,79093291,10400.0,36,7.39,A,A4,5 years,MORTGAGE,60000.0,Source Verified,2016-05-01,Fully Paid,credit_card,741xx,OK,23.70,0.0,1993-09-01,80.0,-999.0,0.0,33.7,56.0,w,0.0,80.0,Individual,0.0,0.0,102.0,79.0,42900.0,0.0,0.0,0.0,2.0,3.0,0.0,4.0,90.9,0.0,0.0,196858.0,83408.0,717.0,59009.0,0.321429,0.625000,0.053571,0.555556,0.300000,0.333333,0.514286,0.555556,0.517857,14.200000,11163.852913,0.199377,0.5,0.011056
6,97567331,6000.0,36,7.49,A,A4,1 year,OWN,25000.0,Source Verified,2017-02-01,Fully Paid,other,941xx,CA,1.20,0.0,2000-06-01,-999.0,-999.0,0.0,1.9,22.0,w,0.0,-999.0,Individual,0.0,0.0,-999.0,2.0,24600.0,1.0,0.0,0.0,9.0,0.0,0.0,1.0,100.0,0.0,0.0,24600.0,0.0,802.0,0.0,0.500000,0.454545,0.045455,0.454545,0.200000,0.090909,0.000000,0.363636,0.181818,12.166667,6397.100123,0.539149,0.5,0.005671
7,69698347,7000.0,36,6.99,A,A3,2 years,MORTGAGE,35000.0,Source Verified,2016-01-01,Fully Paid,debt_consolidation,787xx,TX,6.62,0.0,1990-07-01,-999.0,-999.0,0.0,36.6,14.0,f,0.0,-999.0,Individual,0.0,0.0,107.0,60.0,22500.0